In [3]:
import os
import time
import base64
import json
import requests
import sys
import os
import time
import base64
import json
import requests
import sys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from dotenv import load_dotenv
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException, ElementNotInteractableException
from selenium.webdriver.common.keys import Keys
from function1 import send_text, click_element, set_date_field
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.firefox import GeckoDriverManager  # Added this import












In [ ]:
import os
import time
import base64
import json
import requests
import sys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.firefox import GeckoDriverManager
from dotenv import load_dotenv
from selenium.webdriver.firefox.service import Service
import function1
import automate1

# Get the absolute path to the directory containing this script
script_dir = os.path.dirname(os.path.abspath(__file__))
# Path to the .env file
env_path = os.path.join(script_dir, '.env')
# Check if .env file exists
if not os.path.exists(env_path):
    print(f"ERROR: .env file not found at {env_path}.")
    print("Please create a .env file with the following content:")
    print("TRUECAPTCHA_USER=your_username_here")
    print("TRUECAPTCHA_KEY=your_api_key_here")
    # Use hardcoded credentials as fallback for testing - remove in production
    TRUECAPTCHA_USER = "your_username_here"
    TRUECAPTCHA_KEY = "your_api_key_here"
else:
    # Load TrueCaptcha credentials from .env file
    load_dotenv(dotenv_path=env_path)
    TRUECAPTCHA_USER = os.getenv("TRUECAPTCHA_USER")
    TRUECAPTCHA_KEY = os.getenv("TRUECAPTCHA_KEY")

# Print to help with debugging - we can remove this later
print(f"TrueCaptcha credentials loaded from {env_path}: User={TRUECAPTCHA_USER is not None}, Key={TRUECAPTCHA_KEY is not None}")

def solve_captcha(captcha_img):
    """
    Solve the captcha by taking a screenshot and using TrueCaptcha API.
    Args:
        captcha_img: The WebElement representing the captcha image
    Returns:
        str: The solved captcha text
    """
    # Check if credentials are available
    if not TRUECAPTCHA_USER or not TRUECAPTCHA_KEY:
        error_msg = "TrueCaptcha credentials are missing. Please check your .env file."
        print(error_msg)
        raise ValueError(error_msg)
    
    # Create screenshots directory if it doesn't exist
    os.makedirs("screenshots", exist_ok=True)
    captcha_path = os.path.join("screenshots", "captcha.png")
    
    # Take screenshot of the captcha
    captcha_img.screenshot(captcha_path)
    
    # Base64-encode the screenshot
    with open(captcha_path, "rb") as f:
        b64 = base64.b64encode(f.read()).decode()
    
    # Send to TrueCaptcha API
    resp = requests.post(
        "https://api.apitruecaptcha.org/one/gettext",
        json={"userid": TRUECAPTCHA_USER, "apikey": TRUECAPTCHA_KEY, "data": b64}
    )
    resp.raise_for_status()
    
    # Return the solved captcha text
    return resp.json().get("result", "").strip()


def initialize_browser(firefox_profile_path):
    """
    Initialize Firefox browser with specified profile path
    Args:
        firefox_profile_path: Path to the Firefox profile to use
    Returns:
        WebDriver: Initialized Firefox WebDriver instance
    """
    print(f"Initializing browser with profile: {firefox_profile_path}")
    firefox_options = Options()
    if os.path.exists(firefox_profile_path):
        print(f"Using existing Firefox profile: {firefox_profile_path}")
        firefox_options.add_argument('-profile')
        firefox_options.add_argument(firefox_profile_path)
    else:
        print(f"Profile not found at {firefox_profile_path}")
        print("Using a new profile instead.")
    
    # Automatically download and set up geckodriver
    try:
        print("Setting up geckodriver using webdriver-manager...")
        driver_path = GeckoDriverManager().install()
        print(f"Geckodriver installed at: {driver_path}")
        
        # Initialize the driver with the Service object
        service = Service(driver_path)
        driver = webdriver.Firefox(service=service, options=firefox_options)
        driver.maximize_window()
        return driver
    except Exception as e:
        print(f"Error setting up geckodriver: {e}")
        raise


def perform_login(driver=None, close_after_login=False):
    """
    Perform login to MCA portal.
    Args:
        driver: Existing WebDriver instance (optional). If not provided, will create a new one.
        close_after_login: Whether to close the browser after login.
    Returns:
        tuple: (driver, success) where driver is the WebDriver instance and success is a boolean
               indicating if login was successful
    """
    own_driver = driver is None
    try:
        # Load profile path from config_data.json
        with open("config_data.json", "r") as f:
            config = json.load(f)
        
        # Initialize browser if needed
        if own_driver:
            firefox_profile_path = config["firefox_profile_path"]
            driver = initialize_browser(firefox_profile_path)
        
        # First try to access Fillip page directly
        fillip_url = "https://www.mca.gov.in/content/mca/global/en/mca/llp-e-filling/Fillip.html"
        print("\nAttempting to access Fillip page directly...")
        driver.get(fillip_url)
        
        # Wait for page to load
        WebDriverWait(driver, 20).until(
            lambda d: d.execute_script('return document.readyState') == 'complete'
        )
        
        # Check if we're already logged in
        try:
            time.sleep(5)
            current_url = driver.current_url
            
            if fillip_url in current_url:
                print("\n" + "="*50)
                print("ALREADY LOGGED IN! ACCESSING FILLIP PAGE DIRECTLY.")
                print("="*50 + "\n")
                return driver, True
            else:
                print("Not logged in. Proceeding with login process...")
        except Exception as e:
            print(f"Error checking login status: {e}")
            print("Proceeding with login process...")
        
        # Navigate to login page
        print("\nNavigating to login page...")
        driver.get("https://www.mca.gov.in/content/mca/global/en/foportal/fologin.html")
        
        # Wait for page to load completely
        WebDriverWait(driver, 20).until(
            lambda d: d.execute_script('return document.readyState') == 'complete'
        )
        print("Page load completed")
        time.sleep(5)  # Additional wait for dynamic content
        
        # Handle the dialog box
        print("Looking for dialog box...")
        try:
            ok_button = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//button[@class='btn btn-primary' and @data-dismiss='modal' and contains(text(), 'OK')]"))
            )
            print("Dialog found. Clicking OK button...")
            ok_button.click()
            time.sleep(2)
            print("Dialog closed successfully")
        except Exception as e:
            print(f"No dialog detected or couldn't click OK button: {e}")
            print("Continuing with login process...")
        
        # Wait for login form
        print("Waiting for login form...")
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "form"))
        )
        
        # Fill in User ID
        print("Entering user ID...")
        user_field = None
        for selector in [
            "//input[@type='text']",
            "//input[contains(@placeholder,'User ID')]",
            "//input[contains(@name,'userId')]",
            "//input[@id='userId']",
            "//input[@class='form-control']"
        ]:
            try:
                user_field = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.XPATH, selector))
                )
                if user_field.is_displayed() and user_field.is_enabled():
                    user_field.clear()
                    user_field.send_keys("shagun@registerkaro.in")
                    print("User ID entered successfully")
                    break
            except:
                continue
        
        if not user_field:
            raise Exception("Could not find user ID field")
        
        # Fill in Password
        print("Entering password...")
        try:
            password_field = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "input[type='password']"))
            )
            password_field.clear()
            password_field.send_keys("Plushie0w0!")
            print("Password entered successfully")
        except Exception as e:
            raise Exception(f"Could not find or interact with password field: {e}")
        
        # Set expected successful URL
        expected_url = fillip_url
        max_attempts = 10
        attempt = 1
        login_success = False
        
        # Loop until successful login or max attempts reached
        while attempt <= max_attempts and not login_success:
            print(f"\nCaptcha attempt {attempt}/{max_attempts}")
            
            # Find and solve the captcha
            print("Locating captcha image...")
            try:
                # Try multiple selectors for captcha image
                captcha_selectors = [
                    "img[src*='captcha']",
                    "img[alt*='captcha']",
                    "img[class*='captcha']",
                    "img[src*='Captcha']",
                    "img[alt*='Captcha']",
                    "img[class*='Captcha']"
                ]
                
                captcha_img = None
                for selector in captcha_selectors:
                    try:
                        captcha_img = WebDriverWait(driver, 5).until(
                            EC.presence_of_element_located((By.CSS_SELECTOR, selector))
                        )
                        if captcha_img.is_displayed():
                            print(f"Found captcha image using selector: {selector}")
                            break
                    except:
                        continue
                
                if not captcha_img:
                    raise Exception("Could not find visible captcha image")
                
            except Exception as e:
                print(f"Error finding captcha image: {e}")
                attempt += 1
                continue
            
            # Solve the captcha
            print("Solving captcha...")
            try:
                captcha_text = solve_captcha(captcha_img)
                print(f"Captcha solved: {captcha_text}")
            except Exception as e:
                print(f"Error solving captcha: {e}")
                attempt += 1
                continue
            
            # Enter the solved captcha
            print("Entering captcha solution...")
            try:
                captcha_field = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.ID, "customCaptchaInput"))
                )
                captcha_field.clear()
                captcha_field.send_keys(captcha_text)
                print("Captcha entered successfully")
            except Exception as e:
                print(f"Error entering captcha: {e}")
                attempt += 1
                continue
            
            # Click Login button
            print("Clicking login button...")
            try:
                login_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'Login') or @type='submit']"))
                )
                login_button.click()
                print("Login button clicked")
                
                # Wait for navigation and check for successful login
                print("Waiting for login response...")
                time.sleep(5)

                # Check if we've been redirected to Application History URL
                application_history_url = "https://www.mca.gov.in/content/mca/global/en/application-history.html"
                current_url = driver.current_url
                if application_history_url in current_url:
                    login_success = True
                    print("\n" + "="*50)
                    print("LOGIN SUCCESSFUL! Redirected to Application History page.")
                    print(f"Current URL: {current_url}")
                    print("="*50 + "\n")
                    print("Redirecting to Fillip page...")
                    driver.get(fillip_url)
                    WebDriverWait(driver, 20).until(
                        lambda d: d.execute_script('return document.readyState') == 'complete'
                    )
                    print("Redirected to Fillip page after Application History page.")
                    # Double-check if now on Fillip page
                    if fillip_url in driver.current_url:
                        print("\n" + "="*50)
                        print("SUCCESSFULLY REDIRECTED TO FILLIP PAGE!")
                        print(f"Current URL: {driver.current_url}")
                        print("="*50 + "\n")
                        return driver, True
                    else:
                        print(f"\nWarning: Not on Fillip page after redirect. Current URL: {driver.current_url}")
                        return driver, False
                elif fillip_url in current_url:
                    login_success = True
                    print("\n" + "="*50)
                    print("LOGIN SUCCESSFUL! Already on Fillip page.")
                    print(f"Current URL: {current_url}")
                    print("="*50 + "\n")
                    return driver, True
                # If not redirected, check for error message
                else:
                    try:
                        error_msg = driver.find_element(By.CSS_SELECTOR, ".alert-danger")
                        print(f"Login failed. Error message: {error_msg.text}")
                    except:
                        print("Login failed. No specific error message found.")

                    # Only try to refresh captcha if login failed and we haven't reached max attempts
                    if attempt < max_attempts:
                        print("Refreshing captcha for next attempt...")
                        try:
                            refresh_button = WebDriverWait(driver, 10).until(
                                EC.element_to_be_clickable((By.ID, "refresh-img"))
                            )
                            refresh_button.click()
                            print("Captcha refreshed successfully")
                            time.sleep(2)
                        except Exception as e:
                            print(f"Error refreshing captcha: {e}")
                            # Do not refresh the page, just log and continue

                    attempt += 1
                    if attempt <= max_attempts:
                        print("Retrying with a new captcha...")
                        time.sleep(2)
            except Exception as e:
                print(f"Error during login attempt: {e}")
                attempt += 1
                continue
        
        if not login_success:
            print(f"Failed to login after {max_attempts} attempts.")
            return driver, False
        
        # After successful login, verify we're on Fillip page or Application History page
        current_url = driver.current_url
        application_history_url = "https://www.mca.gov.in/content/mca/global/en/application-history.html"
        if fillip_url in current_url:
            print("\n" + "="*50)
            print("SUCCESSFULLY ON FILLIP PAGE!")
            print(f"Current URL: {current_url}")
            print("="*50 + "\n")
            return driver, True
        elif "/application-history.html" in current_url:
            print("\nDetected Application History page after login. Login successful!")
            print(f"Current URL: {current_url}")
            print("Redirecting to Fillip page...")
            driver.get(fillip_url)
            WebDriverWait(driver, 20).until(
                lambda d: d.execute_script('return document.readyState') == 'complete'
            )
            print("Redirected to Fillip page after Application History page.")
            # Double-check if now on Fillip page
            if fillip_url in driver.current_url:
                print("\n" + "="*50)
                print("SUCCESSFULLY REDIRECTED TO FILLIP PAGE!")
                print(f"Current URL: {driver.current_url}")
                print("="*50 + "\n")
                return driver, True
            else:
                print(f"\nWarning: Not on Fillip page after redirect. Current URL: {driver.current_url}")
                return driver, False
        else:
            print(f"\nWarning: Not on Fillip or Application History page. Current URL: {current_url}")
            return driver, False
            
    except Exception as e:
        print(f"Error during process: {e}")
        return driver, False
    finally:
        # Close the browser if requested and we created it
        if close_after_login and own_driver and driver:
            print("Closing browser as requested.")
            driver.quit()
            return None, False

def main():
    """
    Main workflow function that:
    1. Handles login
    2. After successful login, directly runs LLP Registration form automation
    """
    print("\n" + "="*50)
    print("STARTING MCA AUTOMATION WORKFLOW")
    print("="*50 + "\n")
    
    # Step 1: Load configuration
    try:
        with open("config_data.json", "r") as f:
            config = json.load(f)
        print("Configuration loaded successfully")
    except Exception as e:
        print(f"Error loading configuration: {e}")
        print("Creating default configuration")
        config = {
            "firefox_profile_path": os.path.join(os.path.expanduser("~"), 
                                              "AppData", "Roaming", "Mozilla", "Firefox", "Profiles"),
            "fillip_url": "https://www.mca.gov.in/content/mca/global/en/mca/llp-e-filling/Fillip.html"
        }
        with open("config_data.json", "w") as f:
            json.dump(config, f, indent=4)
    
    # Step 2: Perform login
    print("\nStep 1: Performing login...")
    driver, login_success = perform_login(close_after_login=False)
    
    # Step 3: If login successful, proceed with automation
    if login_success:
        print("\nStep 2: Login successful. Proceeding with automation...")
        try:
            # Wait for page to fully load
            time.sleep(5)
            
            # Run the LLP Registration form automation directly
            print("\n" + "="*50)
            print("STARTING LLP REGISTRATION FORM AUTOMATION")
            print("="*50)
            
            # Initialize automate1 with our driver
            automate1.setup_driver(driver)
            
            # Run the LLP form sequence
            success = automate1.run_llp_form_sequence(driver)
            
            if success:
                print("\n" + "="*50)
                print("AUTOMATION COMPLETED SUCCESSFULLY!")
                print("="*50 + "\n")
            else:
                print("\n" + "="*50)
                print("AUTOMATION COMPLETED WITH ERRORS!")
                print("Some steps may not have completed successfully.")
                print("="*50 + "\n")
            
        except Exception as e:
            print(f"\nError during automation: {e}")
            print("\n" + "="*50)
            print("AUTOMATION FAILED!")
            print("Please check the error messages above.")
            print("="*50 + "\n")
    else:
        print("\n" + "="*50)
        print("LOGIN FAILED! Cannot continue with automation.")
        print("Please check the error messages above.")
        print("="*50 + "\n")
    
    # Ask user if they want to close the browser
    keep_open = input("Do you want to keep the browser open? (y/n): ").lower().strip() == 'y'
    if not keep_open and driver:
        print("Closing browser...")
        driver.quit()
    else:
        print("Keeping browser open. You can continue manually.")
        print("The script will exit but the browser will remain open.")

if __name__ == "__main__":
    main()

In [4]:
firefox_options = Options()
service = Service(GeckoDriverManager().install())
driver = webdriver.Firefox(service=service, options=firefox_options)
driver.get("https://www.mca.gov.in/content/mca/global/en/mca/llp-e-filling/Fillip.html")











====== WebDriver manager ======
Current firefox version is 139.0
Get LATEST geckodriver version for 139.0 firefox
Driver [C:\Users\Admin\.wdm\drivers\geckodriver\win64\v0.36.0\geckodriver.exe] found in cache


In [5]:
with open("config_data.json", "r") as f:
    config_data = json.load(f)

In [6]:
from selenium.webdriver.common.by import By
import time
import os
import json

def handle_file_uploads(driver, config_data):
    """
    Handle file uploads in the attachments page
    """
    print("Handling file uploads...")
    
    # File paths from config
    file1_path = os.path.abspath(config_data["form_data"]["file_paths"]["first_file"])
    file2_path = os.path.abspath(config_data["form_data"]["file_paths"]["second_file"])
    
    # Check if the files exist
    if not os.path.exists(file1_path):
        print(f"Warning: File {file1_path} does not exist!")
        raise Exception(f"First file not found: {file1_path}")
    if not os.path.exists(file2_path):
        print(f"Warning: File {file2_path} does not exist!")
        raise Exception(f"Second file not found: {file2_path}")
    
    print(f"Using files:\n{file1_path}\n{file2_path}")
    
    # Get all the Choose File buttons
    choose_file_buttons = driver.find_elements(By.XPATH, "//button[contains(@class, 'guide-fu-attach-button')]")
    print(f"Found {len(choose_file_buttons)} Choose File buttons")
    

    # SECOND FILE UPLOAD - Use the exact XPath provided
    upload_first_file_last_section(driver, file1_path)

    # Upload first file
    upload_second_file_last_section(driver, file2_path)

In [7]:


def upload_first_file_last_section(driver, file_path):
    """
    Upload file with enhanced input element detection
    """
    print("\n---DOUBLE FILE UPLOAD OF LAST SECTION ---")
    try:
        # Use the exact XPath provided for the Choose File button
        exact_xpath = "/html/body/div[2]/div/div/div/div/div/form/div[4]/div/div[2]/div/div/div[1]/div/div[6]/div/div/div/div[1]/div/div[8]/div/div/div/div[1]/div/div[2]/div/div/div/div[1]/div/div[2]/div/div/div/div[1]/div/div[5]/div/div/div[2]/div[1]/input[1]"
        print(f"Using exact XPath for Choose File button: {exact_xpath}")
        
        # Find the button using the exact XPath
        try:
            upload_button = driver.find_element(By.XPATH, exact_xpath)
            print("Found upload button using exact XPath")
            
            # Find the associated file input
            # First, get the parent container that contains both button and input
            parent_container = upload_button.find_element(By.XPATH, "./ancestor::div[contains(@class, 'guideFieldWidget afFileUpload fileUpload')]")
            print("Found parent container")
            
            # Now find the input element within this container
            file_input = parent_container.find_element(By.XPATH, ".//input[@type='file']")
            print("Found file input element")
            
            # Get the input element's ID
            input_id = file_input.get_attribute('id')
            print(f"Found input element ID: {input_id}")
            
            # Make the file input visible and enabled
            print("Making file input visible and enabled...")
            driver.execute_script("""
                arguments[0].style.display = 'block';
                arguments[0].style.visibility = 'visible';
                arguments[0].style.opacity = '1';
                arguments[0].disabled = false;
                arguments[0].setAttribute('class', 'visible');
            """, file_input)
            time.sleep(2)  # Give more time for the element to become interactive
            
            # Send the file path
            print(f"Sending file path: {file_path}")
            file_input.send_keys(file_path)
            print("File path sent to input element")
            time.sleep(3)
            
            # Verify upload by checking for filename in UI
            try:
                # Extract just the filename from the path for verification
                filename = os.path.basename(file_path)
                
                # Look for the filename text
                filename_element = driver.find_element(By.XPATH, f"//*[contains(text(), '{filename}')]")
                print("SUCCESS: File appears to have uploaded successfully")
            except:
                print("WARNING: File may not have uploaded - no confirmation in UI")
                
                # Try a more direct approach using the ID we found
                print("Trying alternative approach with ID...")
                try:
                    # Try to find the input using its ID
                    input_by_id = driver.find_element(By.ID, input_id)
                    print(f"Found input element by ID: {input_id}")
                    
                    # Make it visible and enabled
                    driver.execute_script("""
                        arguments[0].style.display = 'block';
                        arguments[0].style.visibility = 'visible';
                        arguments[0].style.opacity = '1';
                        arguments[0].disabled = false;
                        arguments[0].setAttribute('class', 'visible');
                    """, input_by_id)
                    time.sleep(2)
                    
                    # Send file path
                    input_by_id.send_keys(file_path)
                    print("File path sent using ID-based approach")
                    time.sleep(3)
                    
                    # Check if it worked
                    try:
                        filename_element = driver.find_element(By.XPATH, f"//*[contains(text(), '{filename}')]")
                        print("SUCCESS: File uploaded with ID-based approach")
                    except:
                        print("WARNING: ID-based approach also failed")
                        print("Please manually upload the file")
                        input("Press Enter after manually uploading the file...")
                except Exception as id_error:
                    print(f"Error with ID-based approach: {id_error}")
                    print("Please manually upload the file")
                    input("Press Enter after manually uploading the file...")
        except Exception as e:
            print(f"Error with XPath approach: {e}")
            
            # Try a completely different approach - JavaScript execution
            print("Trying JavaScript approach...")
            try:
                # Use JavaScript to find and interact with the file input
                script = """
                // Find all file inputs
                var fileInputs = document.querySelectorAll('input[type="file"]');
                
                // Make sure we have at least one file input
                if (fileInputs.length >= 1) {
                    // Get the first file input
                    var input = fileInputs[0];
                    
                    // Make it visible and enabled
                    input.style.display = 'block';
                    input.style.visibility = 'visible';
                    input.style.opacity = '1';
                    input.disabled = false;
                    
                    // Return the element so we can use it
                    return input;
                }
                return null;
                """
                
                # Execute the script
                file_input = driver.execute_script(script)
                
                if file_input:
                    print("Found file input using JavaScript")
                    
                    # Send the file path
                    file_input.send_keys(file_path)
                    print("File path sent using JavaScript approach")
                    time.sleep(3)
                else:
                    print("JavaScript approach failed to find file input")
                    print("Please manually upload the file")
                    input("Press Enter after manually uploading the file...")
            except Exception as js_error:
                print(f"JavaScript approach failed: {js_error}")
                print("Please manually upload the file")
                input("Press Enter after manually uploading the file...")
    except Exception as e:
        print(f"Error with file upload: {e}")
        print("Please manually upload the file")
        input("Press Enter after manually uploading the file...") 





def upload_second_file_last_section(driver, file_path):
    """
    Upload file with enhanced input element detection
    """
    print("\n---DOUBLE FILE UPLOAD OF LAST SECTION ---")
    try:
        # Use the exact XPath provided for the Choose File button
        exact_xpath = "/html/body/div[2]/div/div/div/div/div/form/div[4]/div/div[2]/div/div/div[1]/div/div[6]/div/div/div/div[1]/div/div[8]/div/div/div/div[1]/div/div[2]/div/div/div/div[1]/div/div[2]/div/div/div/div[1]/div/div[9]/div/div/div[2]/div[1]/input[1]"
        print(f"Using exact XPath for Choose File button: {exact_xpath}")
        
        # Find the button using the exact XPath
        try:
            upload_button = driver.find_element(By.XPATH, exact_xpath)
            print("Found upload button using exact XPath")
            
            # Find the associated file input
            # First, get the parent container that contains both button and input
            parent_container = upload_button.find_element(By.XPATH, "./ancestor::div[contains(@class, 'guideFieldWidget afFileUpload fileUpload')]")
            print("Found parent container")
            
            # Now find the input element within this container
            file_input = parent_container.find_element(By.XPATH, ".//input[@type='file']")
            print("Found file input element")
            
            # Get the input element's ID
            input_id = file_input.get_attribute('id')
            print(f"Found input element ID: {input_id}")
            
            # Make the file input visible and enabled
            print("Making file input visible and enabled...")
            driver.execute_script("""
                arguments[0].style.display = 'block';
                arguments[0].style.visibility = 'visible';
                arguments[0].style.opacity = '1';
                arguments[0].disabled = false;
                arguments[0].setAttribute('class', 'visible');
            """, file_input)
            time.sleep(2)  # Give more time for the element to become interactive
            
            # Send the file path
            print(f"Sending file path: {file_path}")
            file_input.send_keys(file_path)
            print("File path sent to input element")
            time.sleep(3)
            
            # Verify upload by checking for filename in UI
            try:
                # Extract just the filename from the path for verification
                filename = os.path.basename(file_path)
                
                # Look for the filename text
                filename_element = driver.find_element(By.XPATH, f"//*[contains(text(), '{filename}')]")
                print("SUCCESS: File appears to have uploaded successfully")
            except:
                print("WARNING: File may not have uploaded - no confirmation in UI")
                
                # Try a more direct approach using the ID we found
                print("Trying alternative approach with ID...")
                try:
                    # Try to find the input using its ID
                    input_by_id = driver.find_element(By.ID, input_id)
                    print(f"Found input element by ID: {input_id}")
                    
                    # Make it visible and enabled
                    driver.execute_script("""
                        arguments[0].style.display = 'block';
                        arguments[0].style.visibility = 'visible';
                        arguments[0].style.opacity = '1';
                        arguments[0].disabled = false;
                        arguments[0].setAttribute('class', 'visible');
                    """, input_by_id)
                    time.sleep(2)
                    
                    # Send file path
                    input_by_id.send_keys(file_path)
                    print("File path sent using ID-based approach")
                    time.sleep(3)
                    
                    # Check if it worked
                    try:
                        filename_element = driver.find_element(By.XPATH, f"//*[contains(text(), '{filename}')]")
                        print("SUCCESS: File uploaded with ID-based approach")
                    except:
                        print("WARNING: ID-based approach also failed")
                        print("Please manually upload the file")
                        input("Press Enter after manually uploading the file...")
                except Exception as id_error:
                    print(f"Error with ID-based approach: {id_error}")
                    print("Please manually upload the file")
                    input("Press Enter after manually uploading the file...")
        except Exception as e:
            print(f"Error with XPath approach: {e}")
            
            # Try a completely different approach - JavaScript execution
            print("Trying JavaScript approach...")
            try:
                # Use JavaScript to find and interact with the file input
                script = """
                // Find all file inputs
                var fileInputs = document.querySelectorAll('input[type="file"]');
                
                // Make sure we have at least one file input
                if (fileInputs.length >= 1) {
                    // Get the first file input
                    var input = fileInputs[0];
                    
                    // Make it visible and enabled
                    input.style.display = 'block';
                    input.style.visibility = 'visible';
                    input.style.opacity = '1';
                    input.disabled = false;
                    
                    // Return the element so we can use it
                    return input;
                }
                return null;
                """
                
                # Execute the script
                file_input = driver.execute_script(script)
                
                if file_input:
                    print("Found file input using JavaScript")
                    
                    # Send the file path
                    file_input.send_keys(file_path)
                    print("File path sent using JavaScript approach")
                    time.sleep(3)
                else:
                    print("JavaScript approach failed to find file input")
                    print("Please manually upload the file")
                    input("Press Enter after manually uploading the file...")
            except Exception as js_error:
                print(f"JavaScript approach failed: {js_error}")
                print("Please manually upload the file")
                input("Press Enter after manually uploading the file...")
    except Exception as e:
        print(f"Error with file upload: {e}")
        print("Please manually upload the file")
        input("Press Enter after manually uploading the file...") 



In [31]:
        # FINAL UPLOADS of file
time.sleep(2)
handle_file_uploads(driver, config_data)
time.sleep(2)
handle_file_uploads(driver, config_data)


Handling file uploads...
Using files:
C:\Users\Admin\Downloads\dummypdf.pdf
C:\Users\Admin\Downloads\dummypdf.pdf
Found 23 Choose File buttons

---DOUBLE FILE UPLOAD OF LAST SECTION ---
Using exact XPath for Choose File button: /html/body/div[2]/div/div/div/div/div/form/div[4]/div/div[2]/div/div/div[1]/div/div[6]/div/div/div/div[1]/div/div[8]/div/div/div/div[1]/div/div[2]/div/div/div/div[1]/div/div[2]/div/div/div/div[1]/div/div[5]/div/div/div[2]/div[1]/input[1]
Found upload button using exact XPath
Found parent container
Found file input element
Found input element ID: guideContainer-rootPanel-panel-panel_1696210624-panel_1548670294-panel-mca_custom_multifile___widget
Making file input visible and enabled...
Sending file path: C:\Users\Admin\Downloads\dummypdf.pdf
File path sent to input element
SUCCESS: File appears to have uploaded successfully

---DOUBLE FILE UPLOAD OF LAST SECTION ---
Using exact XPath for Choose File button: /html/body/div[2]/div/div/div/div/div/form/div[4]/div/di

In [44]:
        # Final form section - add error handling for elements that may not be present
time.sleep(1)
try:
            #*DIN/DPIN/PAN of designated partner
            send_text(driver,css_selector='#guideContainer-rootPanel-panel-panel_1696210624-panel_1548670294-panel-guidetextbox_1527295062___widget', keys="00000001")
except Exception as e:
            print(f"DIN/DPIN/PAN field not found: {str(e)}")

In [45]:
# Declaration and certification by professional
# Enter Name
send_text(driver, css_selector='#guideContainer-rootPanel-panel-panel_1696210624-panel_1548670294-panel-panel_1137005940_cop-guidetextbox___widget', keys=config_data['form_data']['fields']['Enter Name'])
print("success")

success


In [46]:
# Son/Daughter
click_element(driver,css_selector='#guideContainer-rootPanel-panel-panel_1696210624-panel_1548670294-panel-panel_1137005940_cop-guideradiobutton__-1_widget')
print("success")

success


In [47]:
# Enter Father's Name
send_text(driver,css_selector='#guideContainer-rootPanel-panel-panel_1696210624-panel_1548670294-panel-panel_1137005940_cop-guidetextbox_1141998439___widget', keys=config_data['form_data']['fields']["Enter Father's Name"])
print("success")

success


In [48]:
# do state that I am
click_element(driver,css_selector='#guideContainer-rootPanel-panel-panel_1696210624-panel_1548670294-panel-panel_1137005940_cop-guideradiobutton_2109271305__-3_widget')
print("success")

success


In [49]:


# partnership and my membership number
send_text(driver,css_selector='#guideContainer-rootPanel-panel-panel_1696210624-panel_1548670294-panel-panel_1137005940_cop-guidetextbox_1809173783___widget', keys=config_data['form_data']['fields']['partnership and my membership number'])
print("success")

success


In [50]:

# *Whether associate or fellow
click_element(driver,css_selector='#guideContainer-rootPanel-panel-panel_1696210624-panel_1548670294-panel-panel_1137005940_cop-guideradiobutton_598616795__-1_widget')
print("success")

success


In [51]:

# SAVE         
click_element(driver,css_selector='#guideContainer-rootPanel-panel-panel_1696210624-panel_1548670294-panel_copy_copy_copy-mca_button_copy___widget')
print("success")

success


In [52]:

click_element(driver, css_selector='#guideContainer-rootPanel-modal_container_copy-panel_86338280-panel-mca_button___widget')
print("success")

success


In [53]:

click_element(driver, css_selector='#guideContainer-rootPanel-panel-panel_1696210624-panel_1548670294-panel_copy_copy_copy-mca_button___widget')
print("success")

success


In [54]:
# Proceed to Form 9
click_element(driver, css_selector='#guideContainer-rootPanel-panel-panel_1448122692-panel_copy_copy_copy-mca_button___widget')
print("success")

success
